In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from segysak import open_seisnc

from src.definitions import ROOT_DIR
from src.data import make_sua_surfaces

In [ ]:
%matplotlib ipympl

# Load seismic

In [ ]:
# Downloaded files directory
dst_dir = ROOT_DIR / "data"
seisnc_path = dst_dir / "interim/R3136_15UnrPrDMkD_Full_D_Rzn_RMO_Shp_vG.seisnc"

In [ ]:
seisnc = open_seisnc(seisnc_path, chunks={"inline": 100})

# Load Top of Salt

In [ ]:
rnro1_t = make_sua_surfaces.load_mapped_horizon("rnro1_t")

# Load surfaces

In [ ]:
surfaces = xr.open_mfdataset(
    str(ROOT_DIR / "data/processed/surfaces/*.nc"),
    combine="nested",
    concat_dim="anhydrite_perc",
    parallel=True,
    chunks={"anhydrite_perc": -1},
)

In [ ]:
surfaces = surfaces.set_xindex(coord_names="perc")

In [ ]:
surfaces

# Get P10 and P90 surfaces

In [ ]:
dst_dir = ROOT_DIR / "data/processed/summary"
dst = dst_dir / "quantiles.nc"

In [ ]:
if not dst.exists():
    dst_dir.mkdir(parents=True, exist_ok=True)
    quantiles = surfaces.depth.quantile([0.1, 0.25, 0.5, 0.75, 0.9], dim="anhydrite_perc")
    quantiles.to_netcdf(dst)

In [ ]:
quantiles = xr.open_dataarray(dst)  

In [ ]:
quantiles

In [ ]:
inl_sel = 9100

In [ ]:
opt = dict(
    x="xline",
    y="depth",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="Greys",
)

f, ax = plt.subplots(figsize=(16, 10), constrained_layout=True)

seisnc.data.sel(iline=inl_sel, depth=slice(000, 6000)).plot.imshow(ax=ax, **opt)

rnro1_t_trace = rnro1_t.sel(iline=inl_sel)
ax.plot(rnro1_t_trace.xline, rnro1_t_trace.values, label="RNRO1_T")

for q in quantiles["quantile"]:
    q = q.values
    quantile_trace = quantiles.sel(iline=inl_sel, quantile=q)
    ax.plot(quantile_trace.xline, quantile_trace.values, label=f"P{q*100:.0f}")

ax.invert_xaxis()
f.suptitle("RO T surfaces summary")
f.legend(loc='lower center', ncol=9, bbox_to_anchor=(0.5, 0.05))